In [ ]:
from IPython.core.display import HTML
with open ("../../style.css", "r") as file:
    css = file.read()
HTML(css)

# An Interpreter for a Simple Programming Language

In this notebook we develop an interpreter for a small programming language.
The grammar for this language is stored in the file `Pure.g4`.

In [ ]:
!cat -n Pure.g4

The grammar shown above does not contain any actions.  The grammar that is enriched with actions is 
stored in the file `Simple.g4`.

An example program that conforms to this grammar is stored in the file `sum.sl`.

In [ ]:
!cat sum.sl

The file `Simple.g4` contains a parser for the language described by the grammar `Pure.g4`. This parser returns
an abstract syntax tree.  This tree is represented as a *nested tuple*.

In [ ]:
!cat -n Simple.g4

In [ ]:
!antlr4 -Dlanguage=Python3 Simple.g4

In [ ]:
from SimpleLexer  import SimpleLexer
from SimpleParser import SimpleParser
import antlr4

The function `main` takes one parameter `file`.  This parameter is a string specifying a program file.  
The function reads the program contained in this file and executes it.

In [ ]:
def main(file):
    with open(file, 'r') as handle:
        program_text = handle.read()
    parser        = SimpleParser(None)
    input_stream  = antlr4.InputStream(program_text)
    lexer         = SimpleLexer(input_stream)
    token_stream  = antlr4.CommonTokenStream(lexer)
    parser.setInputStream(token_stream)
    result = parser.program()
    execute_list(result.stmnt_list)

The function `execute_list` takes two arguments:
- `Statement_List` is a list of statements,
- `Values` is a dictionary assigning integer values to variable names.

The function executes the statements in `Statement_List`.  If an assignment statement is executed,
the dictionary `Values` is updated.

In [ ]:
def execute_list(Statement_List, Values={}):
    for stmnt in Statement_List:
        execute(stmnt, Values)

The function `execute` takes two arguments:
- `stmnt` is a statement,
- `Values` is a dictionary assigning integer values to variable names.

The function executes the statements in `Statement_List`.  If an assignment statement is executed,
the dictionary `Values` is updated.

In [ ]:
def execute(stmnt, Values):
    op = stmnt[0]
    if op == ':=':
        _, var, value = stmnt
        Values[var] = evaluate(value, Values)
    elif op == 'read':
        _, var = stmnt
        Values[var] = int(input())
    elif op == 'print':
        _, expr = stmnt
        print(evaluate(expr, Values))
    elif op == 'if':
        _, test, SL = stmnt
        if evaluate(test, Values):
            execute_list(SL, Values)
    elif op == 'while':
        _, test, SL = stmnt
        while evaluate(test, Values):
            execute_list(SL, Values)
    else:
        assert False, f'{stmnt} unexpected'

The function `evaluate` takes two arguments:
- `expr` is a logical expression or an arithmetic expression,
- `Values` is a dictionary assigning integer values to variable names.

The function evaluates the given expression and returns this value.

In [ ]:
def evaluate(expr, Values):
    if isinstance(expr, int):
        return expr
    if isinstance(expr, str):
        return Values[expr] 
    op = expr[0]
    if op == '==':
        _, lhs, rhs = expr
        return evaluate(lhs, Values) == evaluate(rhs, Values)
    if op == '<':
        _, lhs, rhs = expr
        return evaluate(lhs, Values) < evaluate(rhs, Values)
    if op == '+':
        _, lhs, rhs = expr
        return evaluate(lhs, Values) + evaluate(rhs, Values)
    if op == '-':
        _, lhs, rhs = expr
        return evaluate(lhs, Values) - evaluate(rhs, Values)
    if op == '*':
        _, lhs, rhs = expr
        return evaluate(lhs, Values) * evaluate(rhs, Values)
    if op == '/':
        _, lhs, rhs = expr
        return evaluate(lhs, Values) / evaluate(rhs, Values)
    assert False, f'{stmnt} unexpected'

In [ ]:
!cat sum.sl

In [ ]:
main('sum.sl')

In [ ]:
!cat factorial.sl

In [ ]:
main('factorial.sl')

In [ ]:
!rm *.py *.tokens *.interp
!rm -r __pycache__/

In [ ]:
!ls